In [2]:
import numpy as np
import pandas as pd
import pickle

In [3]:
with open('hansard-speeches-post2010-lemmatized.pkl', 'rb') as f:
    df = pickle.load(f)

In [15]:
df[['major_heading', 'minor_heading']] = df[['major_heading', 'minor_heading']].fillna(value='')

In [16]:
df['minor_heading'] = df['minor_heading'].str.strip()
df['major_heading'] = df['major_heading'].str.strip()
df['heading'] = df['major_heading'] + ' ' + df['minor_heading']
df['heading'] = df['heading'].str.strip()

In [20]:
df_agg = df.groupby(['date', 'heading', 'display_as']).agg({'speech': lambda x: " ".join(x), 'lemmas': 'sum'})

In [25]:
with open('hansard-speeches-post2010-lemmatized-agg.pkl', 'wb') as f:
    pickle.dump(df_agg, f)